In [25]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

my_drive = '/content/gdrive/MyDrive'
path = os.path.join(my_drive, "AISIA/Paper submission recommendation")
data_dir = os.path.join(path, "")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from tensorflow.python.client import device_lib


In [26]:

import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
# Keras
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Activation, GRU, Bidirectional, MaxPooling1D
from tensorflow.keras.layers import Embedding
# NLTK
import nltk
from nltk.corpus import stopwords
from itertools import product
# Other
import numpy as np
import pandas as pd
import pickle
import re
import string
import math
import pandas as pd
from keras.utils import plot_model

In [ ]:
stop = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "isnt", "it", "its", "itself", "keep", "keeps", "kept", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "names", "named", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "ok", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "puts", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "sees", "serious", "several", "she", "should", "show", "shows", "showed", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]

In [ ]:
# Pre-processing function
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
from nltk.stem import PorterStemmer
st = PorterStemmer()

# Pre-processing function
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

def preprocess_abstract(text):
    text = re.sub("[-]+", " ", text)
    text=text.lower()
    text = re.sub("\d+", "", text)
    text = re.sub("[^\w]", " ", text)

    filtered_sentence = [] 
    for w in text.split(" "): 
        if w not in stop: 
            filtered_sentence.append(w)
    return " ".join(filtered_sentence)
    # return text
    
def preprocess_keywords(text):
    if type(text)==str:
        text=text.lower()
        text = re.sub("\d+", "", text)
        text = re.sub("[^\w]", " ", text)
        text=" ".join(text.split())
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
field = ["Key"]

# Preprocessing for train
df_train=pd.read_csv('/content/gdrive/MyDrive/AISIA/Paper submission recommendation/re_data.csv', encoding = "ISO-8859-1")
df_train = df_train.iloc[df_train[field].dropna().index, :]
df_train["Abstract1"] = df_train["Abstract"].map(preprocess_abstract)
df_train["Title1"] = df_train["Title"].map(preprocess_abstract)
df_train["Key1"] = df_train["Key"].map(preprocess_keywords)
    

# Preprocessing for test data
df_test=pd.read_csv('/content/gdrive/MyDrive/AISIA/Paper submission recommendation/re_data_test.csv', encoding = "ISO-8859-1")
df_test = df_test.iloc[df_test[field].dropna().index, :]
df_test["Abstract"] = df_test["Abstract"].map(preprocess_abstract)
df_test["Title"] = df_test["Title"].map(preprocess_abstract)
df_test["Key"] = df_test["Key"].map(preprocess_keywords)


In [ ]:
df_test


,ID,Title,Abstract,KeyWord,ConferenceID,Key,category,abbr
0,3,covert ephemeral communication data networking,recent years growing belief current ip based ...,NaN,60,security and privacy network security,ACM Conference on Computer and Communications ...,CCS
1,6,automated protocol reverse engineering using s...,network security products nids application fi...,NaN,60,networks network protocols software and its en...,ACM Conference on Computer and Communications ...,CCS
2,9,otit secure provenance modeling location proofs,personal mobile devices location based service...,NaN,60,security and privacy,ACM Conference on Computer and Communications ...,CCS
3,12,privacy outsourced means clustering,attractive organization outsource data analyti...,NaN,60,information systems data management systems da...,ACM Conference on Computer and Communications ...,CCS
4,15,differential privacy neighbourhood spatial...,differential privacy provides strong guarantee...,NaN,60,information systems information systems applic...,ACM Conference on Computer and Communications ...,CCS
...,...,...,...,...,...,...,...,...
4660,14385,linear fitted iteration multiple reward functions,present general detailed development algorithm...,NaN,37,reinforcement learning dynamic programming dec...,Journal of Machine Learning Research,JMLR
4661,14388,sparse unique nonnegative matrix factorization...,nonnegative matrix factorization nmf popular...,NaN,37,nonnegative matrix factorization data preproce...,Journal of Machine Learning Research,JMLR
4662,14391,fast approximation matrix coherence statistica...,statistical leverage scores matrix squar...,NaN,37,matrix coherence statistical leverage randomiz...,Journal of Machine Learning Research,JMLR
4663,14394,regularized bundle methods convex non convex r...,machine learning cast optimization problem id...,NaN,37,optimization non convex non smooth cutting pla...,Journal of Machine Learning Research,JMLR


In [ ]:
df_train["content"] = df_train["Abstract"] + " " + df_train["Key"]  + " " + df_train["Title"]
df_test["content"] = df_test["Abstract"] + " " + df_test["Key"] + " " + df_test["Title"]
#chọn features cần dùng
#df_train["content"] = df_train["Title"]
#df_test["content"] = df_test["Title"]

In [ ]:
vocabulary_size = 50000
tokenizer = Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(df_train["content"])
# Tokenize
sequences = tokenizer.texts_to_sequences(df_train["content"])
data = pad_sequences(sequences)

sequences = tokenizer.texts_to_sequences(df_test["content"])
data_test = pad_sequences(sequences, maxlen=len(data[0]))

# saving
with open('tokenizer_Feature.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
labels1 = {label:index for 
index, label in enumerate(df_train.loc[:, "category"].unique())}
y_train1 = [labels1[value] for index, value in df_train.loc[:, 'category'].iteritems()]
y_test1 = [labels1[value] for index, value in df_test.loc[:, 'category'].iteritems()]

In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)

train_y = np.array(y_train1)
train_y = onehot_encoder.fit_transform(train_y.reshape(len(train_y), 1))

test_y = np.array(y_test1)
test_y = onehot_encoder.transform(test_y.reshape(len(test_y), 1))





#Glove embedding¶


In [27]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [28]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [29]:
downloaded = drive.CreateFile({'id':"1FrRTxsrtkX_NUcXSM23MATHBByEstYcg"})   
downloaded.GetContentFile('glove')


In [30]:

import numpy as np
import codecs
from tqdm.notebook import tqdm

embeddings_index = dict()
f = codecs.open('glove', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))


Loaded 400000 word vectors.


In [31]:

#in 1 từ xem thử
embeddings_index["terribly"]

array([ 0.094505 , -0.5941   ,  0.30503  , -0.42025  ,  0.37319  ,
        0.10674  , -0.16244  ,  0.2722   , -0.032905 , -0.89793  ,
        0.18142  ,  0.011205 , -0.26562  ,  0.088003 , -0.44671  ,
       -0.17606  , -0.45978  , -0.29369  ,  0.14812  , -0.011042 ,
        0.47637  ,  0.52269  ,  0.28908  ,  0.0051047, -0.48634  ,
        0.15869  ,  0.70605  , -0.87434  ,  0.01627  ,  0.49763  ,
       -0.073663 ,  0.5468   , -0.67707  , -0.35679  , -0.45932  ,
        0.23874  , -0.089475 ,  0.0093915,  0.51898  ,  0.04002  ,
        0.1606   , -0.18729  , -0.23099  , -0.51141  , -0.025704 ,
        0.46778  , -0.060693 , -0.35128  ,  0.2131   ,  0.47191  ,
        0.3941   , -0.20462  ,  0.357    ,  0.06115  , -0.52819  ,
        0.07552  ,  0.10749  ,  0.042751 ,  0.72118  ,  0.42015  ,
        0.034104 , -0.34661  ,  0.062814 ,  0.29399  , -0.43359  ,
        0.17527  ,  0.57559  , -0.47333  , -0.079962 , -0.11137  ,
       -0.43098  , -0.45791  , -0.13897  ,  0.52251  ,  0.2881

In [32]:
print('preparing embedding matrix...')
words_not_found = []
embedding_matrix = np.zeros((50000, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector)!=0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))


preparing embedding matrix...
number of null word embeddings: 26161


# Train


In [33]:
def train_keras(model_glove):
    mc = keras.callbacks.ModelCheckpoint(filepath="model_glove_{}.h5".format("_".join(field)), 
                                        monitor='val_accuracy',
                                        mode='max', 
                                        verbose=1, 
                                        save_best_only=True)
    history = model_glove.fit(np.array(data), np.array(train_y),
                        batch_size=896,
                        epochs=10,
                        validation_data=(np.array(data_test),
                                         np.array(test_y)),
                        callbacks=[mc]) 

    X_test=data_test
    y_test=np.array(test_y)
    ## Load lại model tốt nhất đã lưu
    model_glove.load_weights("model_glove_{}.h5".format("_".join(field)))

def accuracy_measure(model_glove):
    y_preds = model_glove.predict(data_test)
    y_tests = np.array(y_test1)
    y_preds = np.array(y_preds)
    for i in [1,3,5,10]:
        y_pred = np.argsort(y_preds, axis=1)[:,-i:]
        a=0
        for j,t in enumerate(y_tests):
            if t in y_pred[j]:
                a=a+1
        print(a/len(y_tests))

In [ ]:
import functools
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy,
                             k=3)
top3_acc.__name__ = "top3"
top5_acc = functools.partial(keras.metrics.top_k_categorical_accuracy,
                             k=5)
top5_acc.__name__ = "top5"
top10_acc = functools.partial(keras.metrics.top_k_categorical_accuracy,
                              k=10)
top10_acc.__name__ = "top10"

import time

t=time.time()
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 300, 
                              input_length=len(data[0]), 
                              weights=[embedding_matrix], 
                              trainable=False))
model_glove.add(Conv1D(1800, 2, activation='relu'))

model_glove.add(GlobalMaxPooling1D())
model_glove.add(Flatten())
model_glove.add(Dense(1000, activation='relu'))
model_glove.add(Dropout(0.2))

model_glove.add(Dense(66, activation='softmax'))

model_glove.compile(loss='categorical_crossentropy', 
                        optimizer="Adam", 
                        metrics=["accuracy",top3_acc,top5_acc,top10_acc])
                        
model_glove.summary()
train_keras(model_glove)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 592, 300)          15000000  
_________________________________________________________________
conv1d (Conv1D)              (None, 591, 1800)         1081800   
_________________________________________________________________
global_max_pooling1d (Global (None, 1800)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1800)              0         
_________________________________________________________________
dense (Dense)                (None, 1000)              1801000   
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 66)                6

In [ ]:

accuracy_measure(model_glove)
#acuuracy lần lượt là top1, top3, top5, top10